## Package

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import os.path
from pathlib import Path

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import plot_model
from tensorflow.keras import regularizers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
#from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import glorot_uniform
#import visualkeras
from tensorflow.keras.optimizers import Adam

from IPython.display import Markdown, display
import seaborn as sns

# from lib import proc_img
# from lib import create_gen

def printmd(string):
    #Print with Markdowns    
    display(Markdown(string))

In [2]:
%load_ext autotime
RANDOM_STATE = 42

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
seed = 7
np.random.seed(seed)

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
tf.test.is_gpu_available()

In [4]:
tf.test.is_built_with_cuda()

True

In [3]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [4]:
x_train, x_valid, y_train, y_valid = train_test_split(train_images, train_labels, test_size=0.2, shuffle= True)

In [5]:
with tf.device('/device:GPU:1'):

    def identity_block(X, f, filters, stage, block):

        conv_name_base = 'res' + str(stage) + block + '_branch'
        bn_name_base = 'bn' + str(stage) + block + '_branch'
        F1, F2, F3 = filters

        X_shortcut = X

        X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
        X = Activation('relu')(X)

        X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
        X = Activation('relu')(X)

        X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

        X = Add()([X, X_shortcut])# SKIP Connection
        X = Activation('relu')(X)

        return X

In [6]:
with tf.device('/device:GPU:1'):

    def convolutional_block(X, f, filters, stage, block, s=2):

        conv_name_base = 'res' + str(stage) + block + '_branch'
        bn_name_base = 'bn' + str(stage) + block + '_branch'

        F1, F2, F3 = filters

        X_shortcut = X

        X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
        X = Activation('relu')(X)

        X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
        X = Activation('relu')(X)

        X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

        X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
        X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

        X = Add()([X, X_shortcut])
        X = Activation('relu')(X)

        return X

In [8]:

dense_sizes_1 = [64,128,256,512]
dense_sizes_2 = [64,128,256,512]
learning_rates_1 = [1e-1,1e-2, 1e-3, 1e-4, 1e-5]

s1_filter_sizes_1 = [16,32,64,128,256,512]

s2_filter_sizes_1 = [16,32,64,128,256,512]
s2_filter_sizes_2 = [16,32,64,128,256,512]
s2_filter_sizes_3 = [128,256,512,1024,2048]

s3_filter_sizes_1 = [16,32,64,128,256,512]
s3_filter_sizes_2 = [16,32,64,128,256,512]
s3_filter_sizes_3 = [128,256,512,1024,2048]

s4_filter_sizes_1 = [16,32,64,128,256,512]
s4_filter_sizes_2 = [16,32,64,128,256,512]
s4_filter_sizes_3 = [128,256,512,1024,2048]

s5_filter_sizes_1 = [16,32,64,128,256,512]
s5_filter_sizes_2 = [16,32,64,128,256,512]
s5_filter_sizes_3 = [128,256,512,1024,2048]

dp_sizes_1 = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]
dp_sizes_2 = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]

wd_sizes_1=[0.0,0.1, 0.001, 0.0001]
wd_sizes_2=[0.0,0.1, 0.001, 0.0001]

opt_layer_1=0
opt_layer_2=0
opt_layer_3=0
opt_layer_4=0
opt_layer_5=0
opt_layer_6=0
opt_layer_7=0
opt_layer_8=0
opt_layer_9=0
opt_layer_10=0
opt_layer_11=0
opt_layer_12=0
opt_layer_13=0
opt_layer_14=0
opt_layer_15=0
opt_layer_16=0
opt_layer_17=0
opt_layer_18=0
opt_layer_19=0
opt_layer_20=0

temp=0
test_acc=0
combinations=0
flag=0
for wd_size_2 in wd_sizes_2 :
    for wd_size_1 in wd_sizes_1 :
        for dp_size_2 in dp_sizes_2 :
            for dp_size_1 in dp_sizes_1 :
                for s5_filter_size_3 in s5_filter_sizes_3:
                    for s5_filter_size_2 in s5_filter_sizes_2:
                        for s5_filter_size_1 in s5_filter_sizes_1:
                            for s4_filter_size_3 in s4_filter_sizes_3:
                                for s4_filter_size_2 in s4_filter_sizes_2:
                                    for s4_filter_size_1 in s4_filter_sizes_1 :
                                        for s3_filter_size_3 in s3_filter_sizes_3:
                                            for s3_filter_size_2 in s3_filter_sizes_2:
                                                for s3_filter_size_1 in s3_filter_sizes_1:
                                                    for s2_filter_size_3 in s2_filter_sizes_3:
                                                        for s2_filter_size_2 in s2_filter_sizes_2:
                                                            for s2_filter_size_1 in s2_filter_sizes_1:
                                                                for s1_filter_size_1 in s1_filter_sizes_1:
                                                                    for dense_size_2 in dense_sizes_2:
                                                                        for dense_size_1 in dense_sizes_1:
                                                                            for learning_rate_1 in learning_rates_1:
                                                                                print(learning_rate_1,dense_size_1,dense_size_2,s1_filter_size_1,s2_filter_size_1,s2_filter_size_2,s2_filter_size_3,s3_filter_size_1,s3_filter_size_2,s3_filter_size_3,s4_filter_size_1,s4_filter_size_2,s4_filter_size_3,s5_filter_size_1,s5_filter_size_2,s5_filter_size_3,dp_size_1,dp_size_2,wd_size_1,wd_size_2)
                                                                                input_shape=(32, 32, 3)
                                                                                X_input = Input(input_shape)

                                                                                X = ZeroPadding2D((1, 1))(X_input)

                                                                                X = Conv2D(s1_filter_size_1, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
                                                                                X = BatchNormalization(axis=3, name='bn_conv1')(X)
                                                                                X = Activation('relu')(X)
                                                                                X = MaxPooling2D((3, 3), strides=(2, 2))(X)

                                                                                X = convolutional_block(X, f=3, filters=[s2_filter_size_1, s2_filter_size_2, s2_filter_size_3], stage=2, block='a', s=1)
                                                                                X = identity_block(X, 3, [s2_filter_size_1, s2_filter_size_2, s2_filter_size_3], stage=2, block='b')
                                                                                X = identity_block(X, 3, [s2_filter_size_1, s2_filter_size_2, s2_filter_size_3], stage=2, block='c')

                                                                                X = convolutional_block(X, f=3, filters=[s3_filter_size_1, s3_filter_size_2, s3_filter_size_3], stage=3, block='a', s=2)
                                                                                X = identity_block(X, 3, [s3_filter_size_1, s3_filter_size_2, s3_filter_size_3], stage=3, block='b')
                                                                                X = identity_block(X, 3, [s3_filter_size_1, s3_filter_size_2, s3_filter_size_3], stage=3, block='c')
                                                                                X = identity_block(X, 3, [s3_filter_size_1, s3_filter_size_2, s3_filter_size_3], stage=3, block='d')

                                                                                X = convolutional_block(X, f=3, filters=[s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='a', s=2)
                                                                                X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='b')
                                                                                X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='c')
                                                                                X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='d')
                                                                                X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='e')
                                                                                X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='f')

                                                                                X = X = convolutional_block(X, f=3, filters=[s5_filter_size_1, s5_filter_size_2, s5_filter_size_3], stage=5, block='a', s=2)
                                                                                X = identity_block(X, 3, [s5_filter_size_1, s5_filter_size_2, s5_filter_size_3], stage=5, block='b')
                                                                                X = identity_block(X, 3, [s5_filter_size_1, s5_filter_size_2, s5_filter_size_3], stage=5, block='c')

                                                                                X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)

                                                                                X= Dropout(dp_size_1)(X)
                                                                                X = Flatten()(X)
                                                                                X=Dense(dense_size_1, activation='relu',kernel_regularizer=regularizers.l2(wd_size_1), name='fc1',kernel_initializer=glorot_uniform(seed=0))(X)
                                                                                X=Dense(dense_size_2, activation='relu',kernel_regularizer=regularizers.l2(wd_size_2), name='fc2',kernel_initializer=glorot_uniform(seed=0))(X)
                                                                                X= Dropout(dp_size_2)(X)
                                                                                X = Dense(10,activation='softmax', name='fc3',kernel_initializer=glorot_uniform(seed=0))(X)

                                                                                model = Model(inputs=X_input, outputs=X, name='ResNet50')

                                                                                my_callbacks = [ tf.keras.callbacks.EarlyStopping(patience=3)]
                                                                                opt = Adam(learning_rate=learning_rate_1)

                                                                                model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                                                                                                                optimizer=opt,
                                                                                                                metrics=['accuracy'])
                                                                                model.fit(x_train,y_train,
                                                                                          batch_size=64,
                                                                                          epochs=100,
                                                                                          validation_data=(x_valid,y_valid),
                                                                                          callbacks=[my_callbacks])
                                                                                

                                                                                _,test_acc = model.evaluate(x_valid, y_valid)
                                                                                print("Current Acc: ",test_acc)
                                                                                print("Best Acc: ",temp)
                                                                                combinations=combinations+1
                                                                                if flag==0 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_1 = learning_rate_1
                                                                                        temp=test_acc
                                                                                    learning_rates_1 = [opt_layer_1]

                                                                                elif flag==1 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_2 = dense_size_1
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_2 = 64
                                                                                    dense_sizes_1 = [opt_layer_2]

                                                                                elif flag==2 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_3 = dense_size_2
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_3 = 64
                                                                                    dense_sizes_2 = [opt_layer_3]

                                                                                elif flag==3 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_4 = s1_filter_size_1
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_4 = 16
                                                                                    s1_filter_sizes_1 = [opt_layer_4]
                                                                                    
                                                                                elif flag==4 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_5 = s2_filter_size_1
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_5 = 16
                                                                                    s2_filter_sizes_1 = [opt_layer_5]
                                                                                    
                                                                                elif flag==5 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_6 = s2_filter_size_2
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_6 = 16
                                                                                    s2_filter_sizes_2 = [opt_layer_6]
                                                                                    
                                                                                elif flag==6 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_7 = s2_filter_size_3
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_7 = 128
                                                                                    s2_filter_sizes_3 = [opt_layer_7]

                                                                                elif flag==7 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_8 = s3_filter_size_1
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_8 = 16
                                                                                    s3_filter_sizes_1 = [opt_layer_8]
                                                                                    
                                                                                elif flag==8 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_9 = s3_filter_size_2
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_9 = 16
                                                                                    s3_filter_sizes_2 = [opt_layer_9]
                                                                                    
                                                                                elif flag==9 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_10 = s3_filter_size_3
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_10 = 128
                                                                                    s3_filter_sizes_3 = [opt_layer_10]
                                                                                    
                                                                                elif flag==10 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_11 = s4_filter_size_1
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_11 = 16
                                                                                    s4_filter_sizes_1 = [opt_layer_11]
                                                                                    
                                                                                elif flag==11 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_12 = s4_filter_size_2
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_12 = 16
                                                                                    s4_filter_sizes_2 = [opt_layer_9]
                                                                                    
                                                                                elif flag==12 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_13 = s4_filter_size_3
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_13 = 128
                                                                                    s4_filter_sizes_3 = [opt_layer_13]
                                                                                    
                                                                                elif flag==13 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_14 = s5_filter_size_1
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_14 = 16
                                                                                    s5_filter_sizes_1 = [opt_layer_14]
                                                                                    
                                                                                elif flag==14 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_15 = s5_filter_size_2
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_15 = 16
                                                                                    s5_filter_sizes_2 = [opt_layer_15]
                                                                                    
                                                                                elif flag==15 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_16 = s5_filter_size_3
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_16 = 128
                                                                                    s5_filter_sizes_3 = [opt_layer_16]
                                                                                    
                                                                                elif flag==16 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_17 = dp_size_1
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_17 = 0.0
                                                                                    dp_sizes_1 = [opt_layer_17]
                                                                                    
                                                                                elif flag==17 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_18 = dp_size_2
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_18 = 0.0
                                                                                    dp_sizes_2 = [opt_layer_18]
                                                                                    
                                                                                elif flag==18 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_19 = wd_size_1
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_19 = 0.0
                                                                                    wd_sizes_1 = [opt_layer_19]
                                                                                    
                                                                                elif flag==19 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_20 = wd_size_2
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_20 = 0.0
                                                                                    wd_sizes_2 = [opt_layer_20]
                                                                            flag=1
                                                                            test_acc=0
                                                                        flag=2
                                                                        test_acc=0
                                                                    flag=3
                                                                    test_acc=0
                                                                flag=4
                                                                test_acc=0
                                                            flag=5
                                                            test_acc=0
                                                        flag=6
                                                        test_acc=0
                                                    flag=7
                                                    test_acc=0
                                                flag=8
                                                test_acc=0
                                            flag=9
                                            test_acc=0
                                        flag=10
                                        test_acc=0  
                                    flag=11
                                    test_acc=0
                                flag=12
                                test_acc=0
                            flag=13
                            test_acc=0
                        flag=14
                        test_acc=0
                    flag=15
                    test_acc=0
                flag=16
                test_acc=0
            flag=17
            test_acc=0
        flag=18
        test_acc=0
    flag=19
    test_acc=0
    
print("######################################################################################################")
print("Trials: ",combinations)
print("Optimal Validation Accuracy",temp)
print("The Final Optimal Values for all block is : ")
print("learning rate: ",opt_layer_1,"\ndense_size_1: ",opt_layer_2,"\ndense_size_2: ",opt_layer_3,"\ns1_filter_size_1: ",opt_layer_4,"\ns2_filter_size_1: ",opt_layer_5,"\ns2_filter_size_2: ",opt_layer_6,"\ns2_filter_size_3: ",opt_layer_7,"\ns3_filter_size_1: ",opt_layer_8,"\ns3_filter_size_2: ",opt_layer_9,"\ns3_filter_size_3: ",opt_layer_10,"\ns4_filter_size_1: ",opt_layer_11,"\ns4_filter_size_2: ",opt_layer_12,"\ns4_filter_size_3: ",opt_layer_13,"\ns5_filter_size_1: ",opt_layer_14,"\ns5_filter_size_2: ",opt_layer_15,"\ns5_filter_size_3: ",opt_layer_16,"\ndp_size_1: ",opt_layer_17,"\ndp_size_2: ",opt_layer_18,"\nwd_size_1: ",opt_layer_19,"\nwd_size_2: ",opt_layer_20)                      

0.1 64 64 16 16 16 128 16 16 128 16 16 128 16 16 128 0.0 0.0 0.0 0.0
Epoch 1/100
625/625 [==============================] - 18s 22ms/step - loss: 2.4197 - accuracy: 0.1015 - val_loss: 2.3056 - val_accuracy: 0.1006
Epoch 2/100
625/625 [==============================] - 13s 20ms/step - loss: 2.3112 - accuracy: 0.1018 - val_loss: 2.3097 - val_accuracy: 0.1039
Epoch 3/100
625/625 [==============================] - 13s 20ms/step - loss: 2.3113 - accuracy: 0.0996 - val_loss: 2.3175 - val_accuracy: 0.0970
Epoch 4/100
313/313 [==============================] - 3s 8ms/step - loss: 2.3099 - accuracy: 0.0952
Current Acc:  0.09520000219345093
Best Acc:  0
0.01 64 64 16 16 16 128 16 16 128 16 16 128 16 16 128 0.0 0.0 0.0 0.0
Epoch 1/100
625/625 [==============================] - 16s 21ms/step - loss: 1.9212 - accuracy: 0.2471 - val_loss: 3.3489 - val_accuracy: 0.1666
Epoch 2/100
625/625 [==============================] - 12s 20ms/step - loss: 1.6641 - accuracy: 0.3671 - val_loss: 2.4265 - val_accur

625/625 [==============================] - 13s 21ms/step - loss: 1.4676 - accuracy: 0.4641 - val_loss: 1.5388 - val_accuracy: 0.4344
Epoch 16/100
625/625 [==============================] - 12s 20ms/step - loss: 1.4430 - accuracy: 0.4734 - val_loss: 1.5593 - val_accuracy: 0.4335
Epoch 17/100
625/625 [==============================] - 12s 20ms/step - loss: 1.4127 - accuracy: 0.4892 - val_loss: 1.4875 - val_accuracy: 0.4506
Epoch 18/100
625/625 [==============================] - 12s 20ms/step - loss: 1.3808 - accuracy: 0.4995 - val_loss: 1.4683 - val_accuracy: 0.4666
Epoch 19/100
625/625 [==============================] - 13s 21ms/step - loss: 1.3645 - accuracy: 0.5054 - val_loss: 1.4960 - val_accuracy: 0.4577
Epoch 20/100
625/625 [==============================] - 13s 20ms/step - loss: 1.3364 - accuracy: 0.5147 - val_loss: 1.4676 - val_accuracy: 0.4708
Epoch 21/100
625/625 [==============================] - 13s 21ms/step - loss: 1.3218 - accuracy: 0.5218 - val_loss: 1.4533 - val_accuracy

Epoch 40/100
625/625 [==============================] - 13s 21ms/step - loss: 1.9826 - accuracy: 0.2554 - val_loss: 1.9724 - val_accuracy: 0.2586
Epoch 41/100
625/625 [==============================] - 13s 20ms/step - loss: 1.9740 - accuracy: 0.2563 - val_loss: 1.9689 - val_accuracy: 0.2570
Epoch 42/100
625/625 [==============================] - 13s 20ms/step - loss: 1.9638 - accuracy: 0.2611 - val_loss: 1.9566 - val_accuracy: 0.2613
Epoch 43/100
625/625 [==============================] - 13s 21ms/step - loss: 1.9544 - accuracy: 0.2646 - val_loss: 1.9447 - val_accuracy: 0.2658
Epoch 44/100
625/625 [==============================] - 13s 20ms/step - loss: 1.9424 - accuracy: 0.2685 - val_loss: 1.9377 - val_accuracy: 0.2710
Epoch 45/100
625/625 [==============================] - 13s 21ms/step - loss: 1.9342 - accuracy: 0.2727 - val_loss: 1.9330 - val_accuracy: 0.2751
Epoch 46/100
625/625 [==============================] - 13s 21ms/step - loss: 1.9273 - accuracy: 0.2742 - val_loss: 1.9191 -

625/625 [==============================] - 13s 20ms/step - loss: 1.5634 - accuracy: 0.4216 - val_loss: 1.6136 - val_accuracy: 0.3949
Epoch 97/100
625/625 [==============================] - 13s 20ms/step - loss: 1.5572 - accuracy: 0.4236 - val_loss: 1.6072 - val_accuracy: 0.3986
Epoch 98/100
625/625 [==============================] - 13s 21ms/step - loss: 1.5513 - accuracy: 0.4265 - val_loss: 1.6153 - val_accuracy: 0.3994
Epoch 99/100
625/625 [==============================] - 13s 21ms/step - loss: 1.5471 - accuracy: 0.4301 - val_loss: 1.6163 - val_accuracy: 0.3978
Epoch 100/100
313/313 [==============================] - 3s 8ms/step - loss: 1.6048 - accuracy: 0.4031
Current Acc:  0.40310001373291016
Best Acc:  0.6004999876022339
0.001 128 64 16 16 16 128 16 16 128 16 16 128 16 16 128 0.0 0.0 0.0 0.0
Epoch 1/100
625/625 [==============================] - 16s 22ms/step - loss: 2.0103 - accuracy: 0.2278 - val_loss: 1.7972 - val_accuracy: 0.2949
Epoch 2/100
625/625 [========================

Epoch 1/100
625/625 [==============================] - 16s 21ms/step - loss: 2.0210 - accuracy: 0.2329 - val_loss: 2.1009 - val_accuracy: 0.2468
Epoch 2/100
625/625 [==============================] - 12s 20ms/step - loss: 1.7058 - accuracy: 0.3593 - val_loss: 1.9703 - val_accuracy: 0.2902
Epoch 3/100
625/625 [==============================] - 13s 20ms/step - loss: 1.5463 - accuracy: 0.4260 - val_loss: 1.9395 - val_accuracy: 0.3434
Epoch 4/100
625/625 [==============================] - 12s 20ms/step - loss: 1.4454 - accuracy: 0.4737 - val_loss: 1.6669 - val_accuracy: 0.4204
Epoch 5/100
625/625 [==============================] - 13s 21ms/step - loss: 1.3677 - accuracy: 0.5045 - val_loss: 1.4764 - val_accuracy: 0.4706
Epoch 6/100
625/625 [==============================] - 13s 21ms/step - loss: 1.2985 - accuracy: 0.5301 - val_loss: 1.4809 - val_accuracy: 0.4764
Epoch 7/100
625/625 [==============================] - 13s 20ms/step - loss: 1.2313 - accuracy: 0.5623 - val_loss: 1.3694 - val_ac

...

625/625 [==============================] - 13s 21ms/step - loss: 1.1900 - accuracy: 0.5817 - val_loss: 1.2131 - val_accuracy: 0.5711
Epoch 11/100
625/625 [==============================] - 13s 21ms/step - loss: 1.1456 - accuracy: 0.6030 - val_loss: 1.2916 - val_accuracy: 0.5443
Epoch 12/100
625/625 [==============================] - 13s 21ms/step - loss: 1.1027 - accuracy: 0.6149 - val_loss: 1.2325 - val_accuracy: 0.5741
Epoch 13/100
313/313 [==============================] - 3s 8ms/step - loss: 1.4600 - accuracy: 0.4952
Current Acc:  0.4952000081539154
Best Acc:  0.6344000101089478
0.001 64 64 16 16 16 128 16 16 128 16 16 128 16 16 128 0.5 0.0 0.0 0.0
Epoch 1/100
625/625 [==============================] - 15s 21ms/step - loss: 2.2160 - accuracy: 0.1586 - val_loss: 2.0117 - val_accuracy: 0.2087
Epoch 2/100
625/625 [==============================] - 13s 21ms/step - loss: 1.8767 - accuracy: 0.2640 - val_loss: 1.8169 - val_accuracy: 0.3038
Epoch 3/100
625/625 [============================

Epoch 21/100
625/625 [==============================] - 13s 21ms/step - loss: 0.9178 - accuracy: 0.6887 - val_loss: 1.1761 - val_accuracy: 0.6019
Epoch 22/100
313/313 [==============================] - 3s 9ms/step - loss: 1.2230 - accuracy: 0.5836
Current Acc:  0.5835999846458435
Best Acc:  0.6344000101089478
0.001 64 64 16 16 16 128 16 16 128 16 16 128 16 16 128 0.8 0.0 0.0 0.0
Epoch 1/100
625/625 [==============================] - 16s 21ms/step - loss: 2.3875 - accuracy: 0.1286 - val_loss: 2.1758 - val_accuracy: 0.1745
Epoch 2/100
625/625 [==============================] - 13s 21ms/step - loss: 2.0650 - accuracy: 0.1903 - val_loss: 2.1457 - val_accuracy: 0.1718
Epoch 3/100
625/625 [==============================] - 13s 20ms/step - loss: 1.9145 - accuracy: 0.2377 - val_loss: 1.8749 - val_accuracy: 0.2449
Epoch 4/100
625/625 [==============================] - 12s 19ms/step - loss: 1.8122 - accuracy: 0.2881 - val_loss: 1.7647 - val_accuracy: 0.3062
Epoch 5/100
625/625 [=================

313/313 [==============================] - 3s 9ms/step - loss: 1.5986 - accuracy: 0.4648
Current Acc:  0.46480000019073486
Best Acc:  0.6344000101089478
0.001 64 64 16 16 16 128 16 16 128 16 16 128 16 16 128 0.0 0.5 0.0 0.0
Epoch 1/100
625/625 [==============================] - 16s 22ms/step - loss: 2.2097 - accuracy: 0.1547 - val_loss: 1.9576 - val_accuracy: 0.2386
Epoch 2/100
625/625 [==============================] - 13s 21ms/step - loss: 1.9193 - accuracy: 0.2637 - val_loss: 1.8584 - val_accuracy: 0.2963
Epoch 3/100
625/625 [==============================] - 13s 21ms/step - loss: 1.7612 - accuracy: 0.3363 - val_loss: 1.6496 - val_accuracy: 0.3687
Epoch 4/100
625/625 [==============================] - 13s 21ms/step - loss: 1.6561 - accuracy: 0.3789 - val_loss: 1.6227 - val_accuracy: 0.3872
Epoch 5/100
625/625 [==============================] - 13s 21ms/step - loss: 1.5814 - accuracy: 0.4173 - val_loss: 1.6085 - val_accuracy: 0.3948
Epoch 6/100
625/625 [==============================

625/625 [==============================] - 13s 20ms/step - loss: 1.8573 - accuracy: 0.2594 - val_loss: 1.8593 - val_accuracy: 0.2785
Epoch 8/100
625/625 [==============================] - 13s 21ms/step - loss: 1.8358 - accuracy: 0.2717 - val_loss: 1.7496 - val_accuracy: 0.3242
Epoch 9/100
625/625 [==============================] - 13s 21ms/step - loss: 1.8047 - accuracy: 0.2859 - val_loss: 1.7779 - val_accuracy: 0.3307
Epoch 10/100
625/625 [==============================] - 14s 22ms/step - loss: 1.7979 - accuracy: 0.2905 - val_loss: 1.7620 - val_accuracy: 0.3122
Epoch 11/100
313/313 [==============================] - 3s 8ms/step - loss: 1.9558 - accuracy: 0.2784
Current Acc:  0.2784000039100647
Best Acc:  0.6344000101089478
0.001 64 64 16 16 16 128 16 16 128 16 16 128 16 16 128 0.0 0.0 0.1 0.0
Epoch 1/100
625/625 [==============================] - 16s 21ms/step - loss: 2.9846 - accuracy: 0.2004 - val_loss: 2.0466 - val_accuracy: 0.2306
Epoch 2/100
625/625 [=============================

Epoch 1/100
625/625 [==============================] - 16s 21ms/step - loss: 2.1663 - accuracy: 0.1934 - val_loss: 2.1759 - val_accuracy: 0.1889
Epoch 2/100
625/625 [==============================] - 13s 21ms/step - loss: 1.7915 - accuracy: 0.3264 - val_loss: 1.7998 - val_accuracy: 0.3363
Epoch 3/100
625/625 [==============================] - 13s 21ms/step - loss: 1.6235 - accuracy: 0.3963 - val_loss: 1.7669 - val_accuracy: 0.3524
Epoch 4/100
625/625 [==============================] - 13s 20ms/step - loss: 1.5078 - accuracy: 0.4457 - val_loss: 1.6562 - val_accuracy: 0.3979
Epoch 5/100
625/625 [==============================] - 13s 21ms/step - loss: 1.4241 - accuracy: 0.4816 - val_loss: 1.4501 - val_accuracy: 0.4686
Epoch 6/100
625/625 [==============================] - 14s 22ms/step - loss: 1.3546 - accuracy: 0.5110 - val_loss: 1.5287 - val_accuracy: 0.4471
Epoch 7/100
625/625 [==============================] - 13s 21ms/step - loss: 1.2842 - accuracy: 0.5386 - val_loss: 1.4342 - val_ac